In [1]:
#r "nuget: Microsoft.SemanticKernel, *-*"
#r "nuget: Microsoft.SemanticKernel.Connectors.AI.HuggingFace, *-*"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.Qdrant, *-*"

Installed Packages Microsoft.SemanticKernel, 1.0.0-beta6 Microsoft.SemanticKernel.Connectors.AI.HuggingFace, 1.0.0-beta6 Microsoft.SemanticKernel.Connectors.Memory.Qdrant, 1.0.0-beta6

In [2]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.Memory.Qdrant;
using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.SemanticKernel.Connectors.AI.HuggingFace.TextEmbedding;

In [3]:
string chat_endpoint = "http://localhost:5002/v1/chat/completions";
string embeddings_endpoint = "http://localhost:5002/v1/embeddings";
string qdrant_endpoint = "http://localhost:6333";

In [4]:
IKernel kernel = new KernelBuilder()
            .WithHuggingFaceTextCompletionService(
                model: "PHI1.5",
                endpoint: chat_endpoint)
            .Build();

In [5]:
var qdrantMemoryBuilder = new MemoryBuilder();

var hfembeddings = new HuggingFaceTextEmbeddingGeneration("jina", embeddings_endpoint);

In [6]:
qdrantMemoryBuilder.WithTextEmbeddingGeneration(hfembeddings);
qdrantMemoryBuilder.WithQdrantMemoryStore(qdrant_endpoint, 768);

In [7]:
var builder = qdrantMemoryBuilder.Build();

In [8]:
string MemoryCollectionName = "jina2demo";

await builder.SaveInformationAsync(MemoryCollectionName, id: "id1", text: "kinfey is Microsoft Cloud Advoate");
await builder.SaveInformationAsync(MemoryCollectionName, id: "id2", text: "kinfey is Microsoft MVP");
await builder.SaveInformationAsync(MemoryCollectionName, id: "id3", text: "kinfey is Microsoft RD");
await builder.SaveInformationAsync(MemoryCollectionName, id: "id4", text: "kinfey is AI Expert");


In [14]:
var searchResults =  builder.SearchAsync(MemoryCollectionName, "do you know kinfey", limit: 3, minRelevanceScore: 0.6);

In [15]:
await foreach (var item in searchResults)
{
    Console.WriteLine(item.Metadata.Text + " : " + item.Relevance);
}

kinfey is AI Expert : 0.84871376
kinfey is Microsoft RD : 0.83069223
kinfey is Microsoft Cloud Advoate : 0.8157256


In [11]:
var questionAnswerFunction = kernel.CreateSemanticFunction(" {{$input}} ");// 

In [12]:
var result = await kernel.RunAsync("do you know Microsoft", questionAnswerFunction);// 

In [13]:
result.GetValue<string>()

 do you know Microsoft 

Answer: Microsoft is a company that makes software and hardware.

Exercise 2: What is a company?

Answer: A company is a group of people who work together to make and sell things.

Exercise 3: What is a consultant?

Answer: A consultant is someone who gives advice to a company.

Exercise 4: What is a real-world example of a company-consultant?

Answer: A real-world example of a company-consultant is a company that hires a consultant to help them improve their customer service.

Exercise 5: What is the role of a consultant in a company?

Answer: The role of a consultant in a company is to give advice and help the company improve.
<|endoftext|>


Question 1: A bakery sells cakes for $25 each. If the bakery sells 15 cakes in a day, how much money does it make in